## Battle of Neighborhoods (Week 1 & Week 2)

#### 1. Introduction

University of Maryland, Baltimore County, UMBC in short, is a public research university in Baltimore County, Maryland. It has 13767 students in 2018 fall enrollment. 

This project will look into the prospect of launching an Korean restaurant near UMBC within 8 miles. The target audience are business people seeking to open a restaurant, or Korean restaurant to be specific. 

Project outline:
1. Location data will be collected from Foursquare
2. Perform EDA 
3. Visualization using folium to locate the best area for a new restaurant.

The best area will be recommended based on criteria as follow:
1. Competitions
2. Surrounding Popular attractions
3. The distance between UMBC and the recommendation location

#### Install and import necessary packages

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    sci

#### 1. Data Collection from Foursquare

### Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'Y5LQ0QRNY2JQNJOJLSJH5TFM2QL4S2BHGECGDLJRTJU2Y0YA' # your Foursquare ID
CLIENT_SECRET = 'GQRFETJAGM3AKDRDBELLQORWUITWGEZP3JA0EEN3J1WZFPJ4' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 200
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y5LQ0QRNY2JQNJOJLSJH5TFM2QL4S2BHGECGDLJRTJU2Y0YA
CLIENT_SECRET:GQRFETJAGM3AKDRDBELLQORWUITWGEZP3JA0EEN3J1WZFPJ4


#### Our objective is to look for possible location around University Maryland - Baltimore County (UMBC) for a Korean Restaurant. Therefore, let's start by converting the UMBC address to its latitude and longitude coordinates.

In [3]:
address = '1000 Hilltop Cir, Baltimore, MD 21250'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

39.25817605 -76.71198838731273


## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

Let's search for Korean restaurants within 8 miles from UMBC.

In [4]:
search_query = 'Korean'
radius = 12874.8
print(search_query + ' .... OK!')

Korean .... OK!


### Define foursquare url

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Y5LQ0QRNY2JQNJOJLSJH5TFM2QL4S2BHGECGDLJRTJU2Y0YA&client_secret=GQRFETJAGM3AKDRDBELLQORWUITWGEZP3JA0EEN3J1WZFPJ4&ll=39.25817605,-76.71198838731273&v=20180604&query=Korean&radius=12874.8&limit=200'

#### Send the GET Request and examine the results

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e58008f69babe001b6970f9'},
 'response': {'venues': [{'id': '54b48705498eb024a85d7dc3',
    'name': 'Brown Rice Korean Grill - N Charles',
    'location': {'address': '2404 N Charles St',
     'lat': 39.31649388430755,
     'lng': -76.61697680303412,
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.31649388430755,
       'lng': -76.61697680303412}],
     'distance': 10447,
     'postalCode': '21218',
     'cc': 'US',
     'city': 'Baltimore',
     'state': 'MD',
     'country': 'United States',
     'formattedAddress': ['2404 N Charles St',
      'Baltimore, MD 21218',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d113941735',
      'name': 'Korean Restaurant',
      'pluralName': 'Korean Restaurants',
      'shortName': 'Korean',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/korean_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '305675',
     'url': 'https://www.

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df_krest = json_normalize(venues)
df_krest.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id,location.crossStreet,location.neighborhood
0,54b48705498eb024a85d7dc3,Brown Rice Korean Grill - N Charles,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",v-1582825676,False,2404 N Charles St,39.316494,-76.616977,"[{'label': 'display', 'lat': 39.31649388430755...",10447,...,"[2404 N Charles St, Baltimore, MD 21218, Unite...",305675,https://www.grubhub.com/restaurant/brown-rice-...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,471719886,NaN,NaN
1,4ba66871f964a5203d4f39e3,Honey Pig Gooldaegee Korean Grill,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",v-1582825676,False,10045 Baltimore National Pike,39.278155,-76.859054,"[{'label': 'display', 'lat': 39.27815455025927...",12868,...,"[10045 Baltimore National Pike, Ellicott City,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4d6aa0466297721ecc55d7b5,Hasung Korean Chinese Restaurant,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",v-1582825676,False,822 N Rolling Rd,39.282827,-76.755043,"[{'label': 'display', 'lat': 39.28282749246845...",4614,...,"[822 N Rolling Rd, Catonsville, MD 21228, Unit...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,51731c69e4b03b46631fcf01,Tako Korean BBQ,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",v-1582825676,False,NaN,39.283731,-76.622569,"[{'label': 'display', 'lat': 39.28373081379521...",8214,...,"[Baltimore, MD, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4ba533b7f964a52040eb38e3,Maryland Korean War Memorial,"[{'id': '4bf58dd8d48988d12d941735', 'name': 'M...",v-1582825676,False,2903 Boston St,39.277514,-76.573532,"[{'label': 'display', 'lat': 39.27751377796241...",12125,...,"[2903 Boston St (Canton Waterfront Park), Balt...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Canton Waterfront Park,NaN


In [8]:
df_krest.describe()

,location.lat,location.lng,location.distance
count,19.000000,19.000000,19.000000
mean,39.284191,-76.704447,9687.473684
std,0.034640,0.104584,2515.744718
min,39.155543,-76.861840,4614.000000
25%,39.278642,-76.822252,9119.000000
50%,39.284012,-76.643803,10172.000000
75%,39.302414,-76.617005,10923.000000
max,39.321777,-76.573532,13123.000000


#### 2. EDA

#### Define information of interest and filter dataframe

In [9]:
filtered_columns = ['name','categories'] + [col for col in df_krest.columns if col.startswith('location.')]+['id']
df_krest_filtered = df_krest.loc[:,filtered_columns]

In [10]:
df_krest_filtered.head()

,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,id
0,Brown Rice Korean Grill - N Charles,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",2404 N Charles St,39.316494,-76.616977,"[{'label': 'display', 'lat': 39.31649388430755...",10447,21218,US,Baltimore,MD,United States,"[2404 N Charles St, Baltimore, MD 21218, Unite...",NaN,NaN,54b48705498eb024a85d7dc3
1,Honey Pig Gooldaegee Korean Grill,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",10045 Baltimore National Pike,39.278155,-76.859054,"[{'label': 'display', 'lat': 39.27815455025927...",12868,21042,US,Ellicott City,MD,United States,"[10045 Baltimore National Pike, Ellicott City,...",NaN,NaN,4ba66871f964a5203d4f39e3
2,Hasung Korean Chinese Restaurant,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",822 N Rolling Rd,39.282827,-76.755043,"[{'label': 'display', 'lat': 39.28282749246845...",4614,21228,US,Catonsville,MD,United States,"[822 N Rolling Rd, Catonsville, MD 21228, Unit...",NaN,NaN,4d6aa0466297721ecc55d7b5
3,Tako Korean BBQ,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",NaN,39.283731,-76.622569,"[{'label': 'display', 'lat': 39.28373081379521...",8214,NaN,US,Baltimore,MD,United States,"[Baltimore, MD, United States]",NaN,NaN,51731c69e4b03b46631fcf01
4,Maryland Korean War Memorial,"[{'id': '4bf58dd8d48988d12d941735', 'name': 'M...",2903 Boston St,39.277514,-76.573532,"[{'label': 'display', 'lat': 39.27751377796241...",12125,21224,US,Baltimore,MD,United States,"[2903 Boston St (Canton Waterfront Park), Balt...",Canton Waterfront Park,NaN,4ba533b7f964a52040eb38e3


In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
# filter the category for each row
df_krest_filtered['categories'] = df_krest_filtered.apply(get_category_type, axis=1)


In [13]:
df_krest_filtered

,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,id
0,Brown Rice Korean Grill - N Charles,Korean Restaurant,2404 N Charles St,39.316494,-76.616977,"[{'label': 'display', 'lat': 39.31649388430755...",10447,21218,US,Baltimore,MD,United States,"[2404 N Charles St, Baltimore, MD 21218, Unite...",NaN,NaN,54b48705498eb024a85d7dc3
1,Honey Pig Gooldaegee Korean Grill,Korean Restaurant,10045 Baltimore National Pike,39.278155,-76.859054,"[{'label': 'display', 'lat': 39.27815455025927...",12868,21042,US,Ellicott City,MD,United States,"[10045 Baltimore National Pike, Ellicott City,...",NaN,NaN,4ba66871f964a5203d4f39e3
2,Hasung Korean Chinese Restaurant,Korean Restaurant,822 N Rolling Rd,39.282827,-76.755043,"[{'label': 'display', 'lat': 39.28282749246845...",4614,21228,US,Catonsville,MD,United States,"[822 N Rolling Rd, Catonsville, MD 21228, Unit...",NaN,NaN,4d6aa0466297721ecc55d7b5
3,Tako Korean BBQ,Korean Restaurant,NaN,39.283731,-76.622569,"[{'label': 'display', 'lat': 39.28373081379521...",8214,NaN,US,Baltimore,MD,United States,"[Baltimore, MD, United States]",NaN,NaN,51731c69e4b03b46631fcf01
4,Maryland Korean War Memorial,Monument / Landmark,2903 Boston St,39.277514,-76.573532,"[{'label': 'display', 'lat': 39.27751377796241...",12125,21224,US,Baltimore,MD,United States,"[2903 Boston St (Canton Waterfront Park), Balt...",Canton Waterfront Park,NaN,4ba533b7f964a52040eb38e3
5,Nak Won Korean Restaurant,Korean Restaurant,12 W 20th St,39.312479,-76.617032,"[{'label': 'display', 'lat': 39.31247942203527...",10172,21218,US,Baltimore,MD,United States,"[12 W 20th St (at Charles St), Baltimore, MD 2...",at Charles St,NaN,4b05796bf964a5209a5822e3
6,Brown Rice Korean Grill,Korean Restaurant,NaN,39.288486,-76.612084,"[{'label': 'display', 'lat': 39.288486, 'lng':...",9246,21202,US,Baltimore,MD,United States,"[Baltimore, MD 21202, United States]",NaN,NaN,5c3cc240f5e9d7002c4aaee2
7,Cham Mat Korean Cuisine,Korean Restaurant,822 N. Rolling Rd.,39.282940,-76.755065,"[{'label': 'display', 'lat': 39.28294, 'lng': ...",4623,21228,US,Catonsville,MD,United States,"[822 N. Rolling Rd., Catonsville, MD 21228, Un...",NaN,NaN,57a8d40f498e0e2f4790427d
8,Koco Korean Fusion,Food Truck,NaN,39.287141,-76.612645,"[{'label': 'display', 'lat': 39.28714124661896...",9148,NaN,US,Baltimore,MD,United States,"[Baltimore, MD, United States]",NaN,NaN,51bf3d22498e44f46a01470f
9,Holy Korean Martyrs Catholic Church,Church,5801 Security Blvd,39.308794,-76.716237,"[{'label': 'display', 'lat': 39.3087942536884,...",5646,21207,US,Woodlawn,MD,United States,"[5801 Security Blvd, Woodlawn, MD 21207, Unite...",NaN,NaN,4d156f5325cda14322a569d6


In [14]:
# clean column names by keeping only last term
df_krest_filtered.columns = [column.split('.')[-1] for column in df_krest_filtered.columns]

In [15]:
### Noticed some categories are not restaurant, we will drop all the unnecessary data.
df_krest_filtered = df_krest_filtered[df_krest_filtered["categories"].str.contains('Restaurant')]


In [16]:
df_krest_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Brown Rice Korean Grill - N Charles,Korean Restaurant,2404 N Charles St,39.316494,-76.616977,"[{'label': 'display', 'lat': 39.31649388430755...",10447,21218,US,Baltimore,MD,United States,"[2404 N Charles St, Baltimore, MD 21218, Unite...",NaN,NaN,54b48705498eb024a85d7dc3
1,Honey Pig Gooldaegee Korean Grill,Korean Restaurant,10045 Baltimore National Pike,39.278155,-76.859054,"[{'label': 'display', 'lat': 39.27815455025927...",12868,21042,US,Ellicott City,MD,United States,"[10045 Baltimore National Pike, Ellicott City,...",NaN,NaN,4ba66871f964a5203d4f39e3
2,Hasung Korean Chinese Restaurant,Korean Restaurant,822 N Rolling Rd,39.282827,-76.755043,"[{'label': 'display', 'lat': 39.28282749246845...",4614,21228,US,Catonsville,MD,United States,"[822 N Rolling Rd, Catonsville, MD 21228, Unit...",NaN,NaN,4d6aa0466297721ecc55d7b5
3,Tako Korean BBQ,Korean Restaurant,NaN,39.283731,-76.622569,"[{'label': 'display', 'lat': 39.28373081379521...",8214,NaN,US,Baltimore,MD,United States,"[Baltimore, MD, United States]",NaN,NaN,51731c69e4b03b46631fcf01
5,Nak Won Korean Restaurant,Korean Restaurant,12 W 20th St,39.312479,-76.617032,"[{'label': 'display', 'lat': 39.31247942203527...",10172,21218,US,Baltimore,MD,United States,"[12 W 20th St (at Charles St), Baltimore, MD 2...",at Charles St,NaN,4b05796bf964a5209a5822e3
6,Brown Rice Korean Grill,Korean Restaurant,NaN,39.288486,-76.612084,"[{'label': 'display', 'lat': 39.288486, 'lng':...",9246,21202,US,Baltimore,MD,United States,"[Baltimore, MD 21202, United States]",NaN,NaN,5c3cc240f5e9d7002c4aaee2
7,Cham Mat Korean Cuisine,Korean Restaurant,822 N. Rolling Rd.,39.282940,-76.755065,"[{'label': 'display', 'lat': 39.28294, 'lng': ...",4623,21228,US,Catonsville,MD,United States,"[822 N. Rolling Rd., Catonsville, MD 21228, Un...",NaN,NaN,57a8d40f498e0e2f4790427d
10,Brown Rice Korean Grill,Korean Restaurant,520 Park Ave,39.296034,-76.618522,"[{'label': 'display', 'lat': 39.2960337970896,...",9090,21201,US,Baltimore,MD,United States,"[520 Park Ave (inside Mt. Vernon Marketplace),...",inside Mt. Vernon Marketplace,NaN,56f2ca60498ea4246d566896
13,Be.bim Korean BBQ,Korean Restaurant,301 W 29th St,39.321777,-76.622126,"[{'label': 'display', 'lat': 39.32177721430469...",10491,NaN,US,Baltimore,MD,United States,"[301 W 29th St (at Remington Ave), Baltimore, ...",at Remington Ave,NaN,584a05d7d25ded7db6a8bb14
14,Tian Chinese & Korean Restaurant,Korean Restaurant,8801 Baltimore National Pike,39.277296,-76.817454,"[{'label': 'display', 'lat': 39.27729618248067...",9335,21043,US,Ellicott City,MD,United States,"[8801 Baltimore National Pike, Ellicott City, ...",NaN,NaN,4bd9badbd2cbc9281397d1ad


#### 3. Visualization

In [17]:
# Visualize the Korean restaurants that are within 8 miles
df_krest_filtered.name

0     Brown Rice Korean Grill - N Charles
1       Honey Pig Gooldaegee Korean Grill
2        Hasung Korean Chinese Restaurant
3                         Tako Korean BBQ
5               Nak Won Korean Restaurant
6                 Brown Rice Korean Grill
7                 Cham Mat Korean Cuisine
10                Brown Rice Korean Grill
13                      Be.bim Korean BBQ
14       Tian Chinese & Korean Restaurant
16     Han Sung Korean / Japanese & Sushi
Name: name, dtype: object

In [18]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around UMBC

# add a red circle marker to represent the UMBC
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='UMBC',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Korean restaurants as blue circle markers
for lat, lng, label in zip(df_krest_filtered.lat, df_krest_filtered.lng, df_krest_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### Let's see how many restaurants within 8 miles of UMBC. 

In [19]:
search_query = 'restaurant'
radius = 12874.8
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results

restaurant .... OK!


{'meta': {'code': 200, 'requestId': '5e57ffd740a7ea001b150bd0'},
 'response': {'venues': [{'id': '4b4e63ecf964a520f2eb26e3',
    'name': "Obrycki's Restaurant & Bar",
    'location': {'address': 'BWI Airport',
     'crossStreet': 'Concourse B',
     'lat': 39.17699488016973,
     'lng': -76.66963395010673,
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.17699488016973,
       'lng': -76.66963395010673}],
     'distance': 9747,
     'postalCode': '21240',
     'cc': 'US',
     'city': 'Baltimore',
     'state': 'MD',
     'country': 'United States',
     'formattedAddress': ['BWI Airport (Concourse B)',
      'Baltimore, MD 21240',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ce941735',
      'name': 'Seafood Restaurant',
      'pluralName': 'Seafood Restaurants',
      'shortName': 'Seafood',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/seafood_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-158282

In [20]:
# assign relevant part of JSON to venues
restaurants = results['response']['venues']

# tranform venues into a dataframe
df_rest = json_normalize(restaurants)
df_rest.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,4b4e63ecf964a520f2eb26e3,Obrycki's Restaurant & Bar,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",v-1582825678,False,BWI Airport,Concourse B,39.176995,-76.669634,"[{'label': 'display', 'lat': 39.17699488016973...",...,MD,United States,"[BWI Airport (Concourse B), Baltimore, MD 2124...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4ad4c017f964a52051f020e3,G&M Restaurant and Lounge,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",v-1582825678,False,804 N Hammonds Ferry Rd,at Nursery Rd,39.224978,-76.665753,"[{'label': 'display', 'lat': 39.22497753291655...",...,MD,United States,"[804 N Hammonds Ferry Rd (at Nursery Rd), Lint...",1546378,https://www.grubhub.com/restaurant/g--m-restau...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN
2,4b2bdf1ef964a520b8bc24e3,Jetro/Restaurant Depot,"[{'id': '58daa1558bbb0b01f18ec1b4', 'name': 'K...",v-1582825678,False,3405 Annapolis Road,NaN,39.247999,-76.638803,"[{'label': 'display', 'lat': 39.24799936531027...",...,MD,United States,"[3405 Annapolis Road, Baltimore, MD 21227, Uni...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4ad4c017f964a5205af020e3,Kibby's Restaurant & Lounge,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1582825678,False,3450 Wilkens Ave,NaN,39.272741,-76.676092,"[{'label': 'display', 'lat': 39.27274108151541...",...,MD,United States,"[3450 Wilkens Ave, Baltimore, MD 21229, United...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4b6718b2f964a520373a2be3,Olive Grove Restaurant & Lounge,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",v-1582825678,False,705 N Hammonds Ferry Rd,NaN,39.222184,-76.665199,"[{'label': 'display', 'lat': 39.22218432877031...",...,MD,United States,"[705 N Hammonds Ferry Rd, Linthicum, MD 21090,...",NaN,NaN,NaN,NaN,NaN,NaN,145808907


In [21]:
df_rest.describe()

,location.lat,location.lng,location.distance
count,50.000000,50.000000,50.000000
mean,39.269759,-76.659402,8229.360000
std,0.046773,0.063078,3194.747844
min,39.166826,-76.825205,1287.000000
25%,39.247683,-76.691379,5566.000000
50%,39.283882,-76.641482,9096.000000
75%,39.293114,-76.616172,10362.250000
max,39.377638,-76.555834,13808.000000


In [22]:
filtered_columns = ['name','categories'] + [col for col in df_rest.columns if col.startswith('location.')]+['id']
df_rest_filtered = df_rest.loc[:,filtered_columns]

In [23]:
# filter the category for each row
df_rest_filtered['categories'] = df_rest_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_rest_filtered.columns = [column.split('.')[-1] for column in df_rest_filtered.columns]

### Noticed some categories are not restaurant, we will drop all the unnecessary data.
df_rest_filtered = df_rest_filtered[df_rest_filtered["categories"].str.contains('Restaurant')]


In [24]:
df_rest_filtered.name

0                Obrycki's Restaurant & Bar
1                 G&M Restaurant and Lounge
3               Kibby's Restaurant & Lounge
4           Olive Grove Restaurant & Lounge
6            Timbuktu Restaurant and Lounge
8            Golden Gate Chinese Restaurant
9               Forno Restaurant + Wine Bar
12                       Pho Van Restaurant
13                        Paul's Restaurant
16                       Shaheen Restaurant
17                    Chesapeake Restaurant
18                        Kumari Restaurant
19                Kumari Restaurant and Bar
20        Pearl's Place Restaurant & Lounge
21                 Penn Restaurant Carryout
22               Mi Casa Mexican Restaurant
23       Mount Everest Restaurant  and  Bar
25                      The Rose Restaurant
26                    St. Mary's Restaurant
27                       Lumbini Restaurant
30        Peppa Flame Restaurant and Lounge
31                      Ban Thai Restaurant
33                Chiapparelli's

In [25]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around UMBC

# add a red circle marker to represent the UMBC
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='UMBC',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the restaurants as blue circle markers
for lat, lng, label in zip(df_rest_filtered.lat, df_rest_filtered.lng, df_rest_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

for lat, lng, label in zip(df_krest_filtered.lat, df_krest_filtered.lng, df_krest_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)


# display map
venues_map

## 4. Explore a popular locations around UMBC
> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**

In [26]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=Y5LQ0QRNY2JQNJOJLSJH5TFM2QL4S2BHGECGDLJRTJU2Y0YA&client_secret=GQRFETJAGM3AKDRDBELLQORWUITWGEZP3JA0EEN3J1WZFPJ4&ll=39.25817605,-76.71198838731273&v=20180604&radius=12874.8&limit=200'

#### Send GET request and examine results

In [27]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 100 around Ecco restaurant.'

#### Get relevant part of JSON

In [28]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4cd9540aa99d3704c3540ace',
  'name': 'Performing Arts and Humanities Building',
  'location': {'address': '1000 Hilltop Cir',
   'crossStreet': 'Hilltop Rd.',
   'lat': 39.255435431376604,
   'lng': -76.71488167988959,
   'labeledLatLngs': [{'label': 'display',
     'lat': 39.255435431376604,
     'lng': -76.71488167988959}],
   'distance': 394,
   'postalCode': '21250',
   'cc': 'US',
   'city': 'Baltimore',
   'state': 'MD',
   'country': 'United States',
   'formattedAddress': ['1000 Hilltop Cir (Hilltop Rd.)',
    'Baltimore, MD 21250',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1ac941735',
    'name': 'College Theater',
    'pluralName': 'College Theaters',
    'shortName': 'Theater',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/performingarts_theater_',
     'suffi

In [29]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Performing Arts and Humanities Building,College Theater,1000 Hilltop Cir,Hilltop Rd.,39.255435,-76.714882,"[{'label': 'display', 'lat': 39.25543543137660...",394,21250,US,Baltimore,MD,United States,"[1000 Hilltop Cir (Hilltop Rd.), Baltimore, MD...",NaN,4cd9540aa99d3704c3540ace
1,Catonsville Family Center Y,Gym,850 S Rolling Rd,NaN,39.252820,-76.727803,"[{'label': 'display', 'lat': 39.25281986111572...",1487,21228,US,Catonsville,MD,United States,"[850 S Rolling Rd, Catonsville, MD 21228, Unit...",NaN,4b4b3528f964a5207f9426e3
2,Cosmic Comix and Toys,Bookstore,87 Mellor Ave,NaN,39.268469,-76.732472,"[{'label': 'display', 'lat': 39.26846916026572...",2104,21228,US,Catonsville,MD,United States,"[87 Mellor Ave, Catonsville, MD 21228, United ...",NaN,4bf412e8ff90c9b6b5cb5428
3,Catonsville Gourmet,Gastropub,829 Frederick Rd,NaN,39.271006,-76.735026,"[{'label': 'display', 'lat': 39.271006, 'lng':...",2445,21228,US,Catonsville,MD,United States,"[829 Frederick Rd, Catonsville, MD 21228, Unit...",NaN,4b057be7f964a520b35822e3
4,Grilled Cheese & Co.,Sandwich Place,500 Edmondson Ave,at Harlem Ln,39.280611,-76.726110,"[{'label': 'display', 'lat': 39.28061140844802...",2778,21228,US,Catonsville,MD,United States,"[500 Edmondson Ave (at Harlem Ln), Catonsville...",NaN,4bb0e005f964a52086653ce3
5,Atwater's Naturally Leavened Bread,Deli / Bodega,815 Frederick Rd,at Winters Lane,39.271257,-76.734446,"[{'label': 'display', 'lat': 39.27125748030575...",2422,21228,US,Catonsville,MD,United States,"[815 Frederick Rd (at Winters Lane), Catonsvil...",NaN,4b62e271f964a5203f572ae3
6,Sprout Natural Choice,Juice Bar,706 Frederick Rd,NaN,39.272193,-76.731760,"[{'label': 'display', 'lat': 39.272193, 'lng':...",2310,21228,US,Catonsville,MD,United States,"[706 Frederick Rd, Catonsville, MD 21228, Unit...",NaN,53b1a72b498e40442f9129fd
7,Patapsco Valley State Park - Avalon Area,State / Provincial Park,5120 South St,NaN,39.227597,-76.725324,"[{'label': 'display', 'lat': 39.22759665742202...",3592,21227,US,Elkridge,MD,United States,"[5120 South St, Elkridge, MD 21227, United Sta...",NaN,4b9ea2aaf964a520acf536e3
8,Guinness Open Gate Brewery & Barrel House,Brewery,5001 Washington Blvd,NaN,39.224665,-76.696676,"[{'label': 'display', 'lat': 39.22466547338667...",3957,21227,US,Halethorpe,MD,United States,"[5001 Washington Blvd, Halethorpe, MD 21227, U...",NaN,59f36841037be12857fe355e
9,Patapsco Valley State Park - Hilton Area,State / Provincial Park,1101 Hilton Ave,NaN,39.245623,-76.746269,"[{'label': 'display', 'lat': 39.24562325600888...",3268,21228,US,Catonsville,MD,United States,"[1101 Hilton Ave, Catonsville, MD 21228, Unite...",NaN,4ccc40e1ee23a1430a6a1da8


In [31]:
dataframe.describe()

,reasons.count,venue.location.lat,venue.location.lng,venue.location.distance,venue.photos.count
count,100.0,100.000000,100.000000,100.000000,100.0
mean,0.0,39.279358,-76.656453,7893.510000,0.0
std,0.0,0.028604,0.063253,2379.357814,0.0
min,0.0,39.202148,-76.816243,394.000000,0.0
25%,0.0,39.271243,-76.706990,7406.000000,0.0
50%,0.0,39.284994,-76.620695,8932.000000,0.0
75%,0.0,39.294223,-76.612745,9424.500000,0.0
max,0.0,39.336500,-76.598809,10474.000000,0.0


In [30]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as yellow circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='yellow',
        fill_color='yellow',
        fill_opacity=0.6
        ).add_to(venues_map)
for lat, lng, label in zip(df_rest_filtered.lat, df_rest_filtered.lng, df_rest_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

for lat, lng, label in zip(df_krest_filtered.lat, df_krest_filtered.lng, df_krest_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### The red marker indicates UMBC, green marker indicates Korean restaurant nearby, blue markers indicates other restaurants competitors, and yellow indicates 

#### Conclusion

According to our result, it is recommended to open a new restaurant/ Korean restaurant around Catonsville and Woodlawn area. It is because the competition between similar type of Korean restaurants and other restaurants are relatively small comparing to downtown Baltimore. Moreover, the Catonsville area has quite a few popular attractions, and the distance is relatively shorter than downtown Baltimore. 

